In [1]:
from script.traj_to_dataframe import *
from script.inverse_design import *
import pickle

In [ ]:
from script.descriptor

# Generate fingerprint dataframe to train

In [2]:
# open dataset.pkl
with open('/home/ahrehd0506/git/New_active_motif/data/CO_docs_slab_added.pkl', 'rb') as p_f:
    data = pickle.load(p_f)


# remove outlier, nitrdie, sulfide, hydrate
Nitride=[]
Sulfide=[]
Hydrate=[]
outliner=[]

Else_index = []
new_data=[]

for i, v in enumerate(data):
    element = v['slab_information']['atoms']['chemical_symbols']
    energy =  v['energy']
    slab_movement = v['movement_data']['max_slab_movement']
    adsorb_movement = v['movement_data']['max_adsorbate_movement']
    if (energy >= -2.3) & (energy <= 0.3) & (slab_movement <= 1.2) & (adsorb_movement <= 1.49):
        outliner.append(i)
        if 'N' in element:
            Nitride.append(i)
        elif 'S' in element:
            Sulfide.append(i)
        elif 'H' in element:
            Hydrate.append(i)
        else:
            Else_index.append(i)
Del_index = Nitride+Sulfide+Hydrate

for i in Else_index:
    new_data.append(data[i])

# collect trajactory name list
new_slab_name = []
for i in new_data:
    name = i['mpid']+'_'+str(i['mongo_id'])
    new_slab_name.append(name)
new_slab_name.sort()

# set path which have atoms.traj
path = '/home/ahrehd0506/research/GASpy/final_CO_slab/'

# make fingerprint
df, error = make_traj_to_dataframe(new_slab_name,path)
df

,name,active_number,active_block,active_radi,active_oxi,active_X,active_row,active_group,active_thermal,active_bp,...,down_radi,down_oxi,down_X,down_row,down_group,down_thermal,down_bp,down_mp,down_IE,down_natom
0,mp-10010_5d83020d30582ea2977a314b,27.0,3.0,0.768333,2.0,1.88,100.00,9.0,4.0,3200.0,...,0.607500,-0.500000,1.755000,192.500000,13.5,3.000000,2982.5,1310.235000,70687.5,2
1,mp-10010_5d83cd3a4eaf9091a4055c72,27.0,3.0,0.768333,2.0,1.88,100.00,9.0,4.0,3200.0,...,0.607500,-0.500000,1.755000,192.500000,13.5,3.000000,2982.5,1310.235000,70687.5,2
2,mp-10010_5d83f778dafe868ae5d44057,27.0,3.0,0.768333,2.0,1.88,100.00,9.0,4.0,3200.0,...,0.607500,-0.500000,1.755000,192.500000,13.5,3.000000,2982.5,1310.235000,70687.5,2
3,mp-10010_5d8688095436ecdede9a240e,14.0,2.0,0.540000,-4.0,1.90,150.00,14.0,3.0,3173.0,...,0.661111,0.333333,1.796667,161.666667,12.0,3.333333,3055.0,1462.823333,73395.0,3
4,mp-10010_5d8688095436ecdede9a2467,27.0,3.0,0.768333,2.0,1.88,100.00,9.0,4.0,3200.0,...,0.540000,-4.000000,1.900000,150.000000,14.0,3.000000,3173.0,1687.000000,81517.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16092,mvc-16102_5d83cd3b4eaf9091a405660c,13.0,2.0,0.675000,3.0,1.61,235.00,13.0,3.0,2792.0,...,1.013333,-2.000000,2.550000,0.520000,16.0,4.000000,958.0,494.000000,97524.0,2
16093,mvc-16102_5d83f779dafe868ae5d46655,34.0,2.0,1.013333,-2.0,2.55,0.52,16.0,4.0,958.0,...,0.711250,2.750000,1.682500,276.250000,12.5,3.250000,2894.0,1039.545000,64209.5,4
16094,mvc-16102_5d83f779dafe868ae5d46656,13.0,2.0,0.675000,3.0,1.61,235.00,13.0,3.0,2792.0,...,1.013333,-2.000000,2.550000,0.520000,16.0,4.000000,958.0,494.000000,97524.0,2
16095,mvc-16102_5d83f779dafe868ae5d46657,34.0,2.0,1.013333,-2.0,2.55,0.52,16.0,4.0,958.0,...,0.675000,3.000000,1.610000,235.000000,13.0,3.000000,2792.0,933.470000,59858.0,2


# Training

In [8]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

# input energy to fingerprint
energy_list = []
name_list =[]
data_energy = []
for i in data:
    name_list.append(str(i['mpid'])+'_'+str(i['mongo_id']))
    energy_list.append(i['energy'])
for name in df['name']:
    for index,value in enumerate(name_list):
        if name == value:
            data_energy.append(energy_list[index])
df['energy'] = data_energy

name = df.loc[:,'name']

X = df.iloc[:,1:-1]
y = df.loc[:,'energy'].astype(np.float)

scalar = StandardScaler()
X = scalar.fit_transform(X.astype(np.float))

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Hyperparameter optimization

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import time
mae_set = []
mae_train_set = []
rmse_set = []
rmse_train_set = []
r2_set = []
r2_train_set = []

#reg = GradientBoostingRegressor()

#search_space = {
#    "learning_rate": Real(0.0000001,1), 
#    "n_estimators": Integer(100,5000),
#    "min_samples_split": Integer(2, 30),
#    "min_samples_leaf": Integer(1,30),
#    "loss": Categorical(['ls', 'lad','huber','quantile']),
#    "max_depth": Integer(1,30),
#    "max_features": Categorical(['auto', 'sqrt','log2']),
#}

#reg_bayes = BayesSearchCV(
#    estimator=reg,
#    search_spaces=search_space,
#    random_state=0,
#    cv=5,
#    verbose=2,
#    n_jobs=-1
#)
#reg_bayes.fit(X_train,y_train)
#learn_rate = reg_bayes.best_estimator_.learning_rate
#n_esti = reg_bayes.best_estimator_.n_estimators
#depth = reg_bayes.best_estimator_.max_depth
#min_leaf = reg_bayes.best_estimator_.min_samples_leaf
#min_split = reg_bayes.best_estimator_.min_samples_split
#max_feat = reg_bayes.best_estimator_.max_features
#losss = reg_bayes.best_estimator_.loss

#print(learn_rate,n_esti,depth,min_leaf,min_split,max_feat,losss)
#hyper_params = {'n_estimators':np.arange(1,1000,1),'subsample':np.linspace(0,1,100),'learning_rate':np.linspace(0,1,100)}
#tune = GradientBoostingRegressor()
#g_search = RandomizedSearchCV(tune, hyper_params, cv=5, n_jobs=-1,random_state=42)
#g_search.fit(X_train,y_train)
#estimators = g_search.best_estimator_.n_estimators
#subsamples = g_search.best_estimator_.subsample
#learning_rates = g_search.best_estimator_.learning_rate

# training 
for i in range(0,1):
    print('start')
    train_start = time.time()
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=i)
    regressor =GradientBoostingRegressor(n_estimators=3938, learning_rate=0.14777,max_depth=17,
                                         max_features='sqrt',min_samples_leaf=28, min_samples_split=24,loss='lad',random_state=42)
    regressor.fit(X_train,y_train)
    train_elapsed = time.time() - train_start
    print(train_elapsed)
    y_pred_train = regressor.predict(X_train)
    y_pred = regressor.predict(X_test)

    print('Score train set: ', regressor.score(X_train,y_train))
    print('Score test set: ',regressor.score(X_test,y_test))
    mae_train_set.append(mean_absolute_error(y_pred_train,y_train))
    rmse_train_set.append(mean_squared_error(y_pred_train,y_train,squared=False))
    mae_set.append(mean_absolute_error(y_pred,y_test))
    rmse_set.append(mean_squared_error(y_pred,y_test,squared=False))
    r2_set.append(r2_score(y_pred,y_test))
    r2_train_set.append(r2_score(y_pred_train,y_train))
    


<ipython-input-8-c6672a7e1464>:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = df.loc[:,'energy'].astype(np.float)
<ipython-input-8-c6672a7e1464>:26: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = scalar.fit_transform(X.astype(np.float))


start
268.1762239933014
Score train set:  0.9520701306937082
Score test set:  0.8954367158729488


# Import unique active motif from dataset

In [1]:
from itertools import permutations
atom_list = []
for name in new_slab_name:
    atom_list.append(read(path+name+'.traj'))
    
active_motif_dict_list = []
for atoms in atom_list:
    active_motif_dict_list.append(get_unique_active_motif(atoms))
    
elements = ['Cu','Al','Si','Pd','Au','Pt','Ga','Sn','In','Ni','Se','Zn','Ge','As','Sb','Fe','V','Rh',
           'Ir','Ag','Co','Re','W','Ti','Pb','Os','Mo','Mn','Ru','Cr']
combi_elements = list(permutations(elements,2))

fingerprint, error = active_motif_to_fingerprint(active_motif_dict_list,combi_elements)


target_predict_dataframe_list =[]
df = pd.DataFrame({'name' : list(new_slab_name)},columns=['name'])
for property in properties:
    column_name_list = ['active','floor','down']
    for column_name, i in zip(column_name_list,range(0,3)):
        df[column_name+'_number'] = property[0][i]
        df[column_name+'_block'] = property[1][i]
        df[column_name+'_radi'] = property[2][i]
        df[column_name+'_oxi'] = property[3][i]
        df[column_name+'_X'] = property[4][i]
        df[column_name+'_row'] = property[5][i]
        df[column_name+'_group'] = property[6][i]
        df[column_name+'_thermal'] = property[7][i]
        df[column_name+'_bp'] =property[8][i]
        df[column_name+'_mp'] = property[9][i]
        df[column_name+'_IE'] = property[10][i] 
        df[column_name+'_natom'] = property[11][i]

    predict_X = df.iloc[:,1:-1]
    predict_X = scalar.fit_transform(predict_X.astype(np.float64))
    H_predict_y = H_regressor.predict(predict_X)
    CO_predict_y = regressor.predict(predict_X)
    energy_df = pd.DataFrame({'name' : df['name'], 'H_energy' :H_predict_y,'CO_energy':CO_predict_y},
                             columns=['name','H_energy','CO_energy'])
    target_predict_dataframe_list.append(energy_df)
    

NameError: name 'new_slab_name' is not defined